In [1]:
import numpy as np
import pandas as pd
import json
import ast
from datetime import datetime
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
df_reviews= pd.read_csv('../archivos_csv/ETL1_steam_games.csv')

In [4]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [5]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [6]:
print(df_reviews['reviews'][0])
print(type(df_reviews['reviews'][0]))

[{'funny': '', 'posted': 'Posted November 5, 2011.', 'last_edited': '', 'item_id': '1250', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}, {'funny': '', 'posted': 'Posted July 15, 2011.', 'last_edited': '', 'item_id': '22200', 'helpful': 'No ratings yet', 'recommend': True, 'review': "It's unique and worth a playthrough."}, {'funny': '', 'posted': 'Posted April 21, 2011.', 'last_edited': '', 'item_id': '43110', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]
<class 'str'>


In [7]:
df_reviews['reviews']= df_reviews['reviews'].apply(lambda x: ast.literal_eval(x))


In [8]:
print(df_reviews['reviews'][0])

[{'funny': '', 'posted': 'Posted November 5, 2011.', 'last_edited': '', 'item_id': '1250', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}, {'funny': '', 'posted': 'Posted July 15, 2011.', 'last_edited': '', 'item_id': '22200', 'helpful': 'No ratings yet', 'recommend': True, 'review': "It's unique and worth a playthrough."}, {'funny': '', 'posted': 'Posted April 21, 2011.', 'last_edited': '', 'item_id': '43110', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]


In [9]:
len(df_reviews)

25799

In [10]:
# desanidamos la columna reviews y la transformamos en un DataFrame

df_reviews2 = pd.json_normalize(df_reviews['reviews'])

# Mostrar los primeros registros del nuevo DataFrame
df_reviews2.head()

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


In [11]:
df_reviews2= pd.concat([df_reviews[['user_id', 'user_url']], df_reviews2], axis= 1)  #concatenamos ambos dataframes, reemplazando la columna reviews por los reviewes de cada usuario

In [12]:
df_reviews2.head(10)

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None
5,Wackky,http://steamcommunity.com/id/Wackky,"{'funny': '', 'posted': 'Posted May 5, 2014.',...","{'funny': '', 'posted': 'Posted December 24, 2...","{'funny': '1 person found this review funny', ...","{'funny': '', 'posted': 'Posted March 20, 2012...","{'funny': '', 'posted': 'Posted March 9, 2012....",None,None,None,None,None
6,76561198079601835,http://steamcommunity.com/profiles/76561198079...,"{'funny': '1 person found this review funny', ...",None,None,None,None,None,None,None,None,None
7,MeaTCompany,http://steamcommunity.com/id/MeaTCompany,"{'funny': '', 'posted': 'Posted July 24.', 'la...",None,None,None,None,None,None,None,None,None
8,76561198089393905,http://steamcommunity.com/profiles/76561198089...,"{'funny': '5 people found this review funny', ...","{'funny': '1 person found this review funny', ...",None,None,None,None,None,None,None,None
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"{'funny': '', 'posted': 'Posted June 16.', 'la...",None,None,None,None,None,None,None,None,None


In [13]:
# transformamos en filas cada una de las columnas desanidadas, tomando como referencia las columnas user_id y user_url
df_reviews2 = pd.melt(df_reviews2, id_vars=['user_id', 'user_url'], value_vars=list(range(9)), value_name='reviews')
df_reviews2.head()

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."


In [14]:
df_reviews2 = df_reviews2.dropna()

In [15]:
df_reviews = df_reviews2['reviews'].apply(pd.Series, dtype='object')
df_reviews = df_reviews.add_prefix('reviews_')
df_reviews = pd.concat([df_reviews2[['user_id', 'user_url']], df_reviews], axis=1)
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [16]:
# vemos que hay registros con casilleros vacios, reemplazamos esos casilleros vacios por el valor None
len(df_reviews['reviews_funny'][0])

0

In [17]:
df_reviews.replace('', None, inplace= True)

In [18]:
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,None,Posted February 3.,None,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,None,"Posted October 14, 2013.",None,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",None,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [19]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59036 entries, 0 to 232129
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              59036 non-null  object
 1   user_url             59036 non-null  object
 2   reviews_funny        8132 non-null   object
 3   reviews_posted       59036 non-null  object
 4   reviews_last_edited  6083 non-null   object
 5   reviews_item_id      59036 non-null  object
 6   reviews_helpful      59036 non-null  object
 7   reviews_recommend    59036 non-null  bool  
 8   reviews_review       59006 non-null  object
dtypes: bool(1), object(8)
memory usage: 6.1+ MB


In [20]:
# existen columnas que cuentan con una gran proporcion de valores nulos/None

nulos_funny = len(df_reviews[df_reviews['reviews_funny'].isnull()])
nulos_last_edited = len(df_reviews[df_reviews['reviews_last_edited'].isnull()])

print(f'Porcentaje de valores de campo "reviewes_funny" nulos sobre el total de registros: {round((nulos_funny / len(df_reviews)) * 100, 2)}%')
print(f'Porcentaje de valores de campo "reviewes_last_edited" nulos sobre el total de registros: {round((nulos_last_edited / len(df_reviews)) * 100, 2)}%')


# como vemos, estos campos cuentan con muy poco proporcion de valores no nulos, por lo que la informacion que aportarian es escasa y provocarian sesgos
# decidimos eliminar dichas columnas


Porcentaje de valores de campo "reviewes_funny" nulos sobre el total de registros: 86.23%
Porcentaje de valores de campo "reviewes_last_edited" nulos sobre el total de registros: 89.7%


In [21]:
df_reviews = df_reviews.drop(columns= ['reviews_funny', 'reviews_last_edited']) 

In [22]:
df_reviews.tail()

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
231919,SKELETRONPRIMEISOP,http://steamcommunity.com/id/SKELETRONPRIMEISOP,"Posted August 15, 2014.",440,No ratings yet,True,TF2 is alot of fun and its really good but the...
231921,76561198141079508,http://steamcommunity.com/profiles/76561198141...,"Posted August 2, 2014.",304930,No ratings yet,True,Fun game with friends
232047,ShadowYT100,http://steamcommunity.com/id/ShadowYT100,"Posted July 31, 2015.",265630,No ratings yet,True,So Fun!! :D
232127,bestcustomurlevermade,http://steamcommunity.com/id/bestcustomurlever...,"Posted December 20, 2015.",304050,No ratings yet,True,"This game is great. The only thing is,Why cant..."
232129,76561198209894493,http://steamcommunity.com/profiles/76561198209...,Posted February 3.,570,0 of 2 people (0%) found this review helpful,False,DOTA 2.normal skill - 2ez for me>high skill - ...


In [23]:
# Posted February 2, 2012.
# Posted November 5, 2011.

In [24]:
# vamos a reiniciar los indices para que no haya valores discontinuos, y facilitar la manipulacion de los datos en el dataframe.
df_reviews= df_reviews.reset_index(drop= True)

In [25]:
# vemos que los registros de la columna 'reviews_posted' no tienen el formato de fecha adecuado, por lo que debemos pasarlo a un formato de fecha que podamos utilizar.
no_formateadas= list()


for idx,fecha in enumerate(df_reviews['reviews_posted']):
    try:
        if ',' in fecha:
            df_reviews['reviews_posted'][idx] = datetime.strptime(fecha, 'Posted %B %d, %Y.').date()
        else:
            no_formateadas.append(df_reviews['reviews_posted'][idx])
            df_reviews['reviews_posted'][idx] = 'formato invalido'
            continue
    except Exception as e:
        continue

C:\Users\usuario\AppData\Local\Temp\ipykernel_8648\1376723300.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews['reviews_posted'][idx] = datetime.strptime(fecha, 'Posted %B %d, %Y.').date()
C:\Users\usuario\AppData\Local\Temp\ipykernel_8648\1376723300.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews['reviews_posted'][idx] = 'formato invalido'


In [26]:
df_reviews['reviews_posted']

0              2011-11-05
1              2014-06-24
2        formato invalido
3              2013-10-14
4              2014-04-15
               ...       
59031          2014-08-15
59032          2014-08-02
59033          2015-07-31
59034          2015-12-20
59035    formato invalido
Name: reviews_posted, Length: 59036, dtype: object

In [27]:
len(df_reviews['reviews_posted'])

59036

In [28]:
no_formateadas

['Posted February 3.',
 'Posted May 20.',
 'Posted July 24.',
 'Posted June 16.',
 'Posted June 11.',
 'Posted February 27.',
 'Posted March 31.',
 'Posted July 9.',
 'Posted June 16.',
 'Posted January 18.',
 'Posted July 30.',
 'Posted February 20.',
 'Posted January 11.',
 'Posted February 26.',
 'Posted August 31.',
 'Posted March 27.',
 'Posted May 26.',
 'Posted January 18.',
 'Posted July 20.',
 'Posted July 23.',
 'Posted September 9.',
 'Posted February 22.',
 'Posted May 29.',
 'Posted June 20.',
 'Posted May 17.',
 'Posted September 17.',
 'Posted September 4.',
 'Posted July 16.',
 'Posted June 14.',
 'Posted March 19.',
 'Posted February 7.',
 'Posted April 10.',
 'Posted September 15.',
 'Posted August 19.',
 'Posted June 22.',
 'Posted March 27.',
 'Posted May 1.',
 'Posted March 7.',
 'Posted January 10.',
 'Posted May 15.',
 'Posted September 8.',
 'Posted September 2.',
 'Posted August 26.',
 'Posted July 18.',
 'Posted May 14.',
 'Posted March 30.',
 'Posted June 18.

In [29]:
# controlamos que los registros que no pudieron ser formateados, presenten todos la palabra 'Posted' como comienzo de la cadena.
control= list(filter(lambda x: str(x).startswith('Posted'), df_reviews['reviews_posted']))
len(control)

0

In [30]:
# controlamos que no hayamos pasado ningun registro con los 3 valores (dd-mm-aaaa) y que tengan formato diferente
# para ello filtramos a traves de una condicion los valores presentes en la lista de cadenas que no pudieron ser formateadas.
# la condicion es que, como cada registro presenta el prefijo 'Posted', si podemos dividir el registro en 4 partes, inferimos que existen registros
# que a causa de su diferente formato de tipeo, no pudieron ser transformados y podriamos haber pasado por alto.

control= list(filter(lambda x: len(x.split()) == 4, no_formateadas))
len(control)

# como el control nos dio que ningun registro cumplio la condicion, entonces los regitros que no pudieron ser convertidos, no disponen del formato adecuado para 
# transformarlos en fecha

0

In [34]:
# eliminamos los valores nulos que hayan quedado en algun registro
df_reviews = df_reviews.dropna()
df_reviews.isna().sum()

user_id              0
user_url             0
reviews_posted       0
reviews_item_id      0
reviews_helpful      0
reviews_recommend    0
reviews_review       0
dtype: int64

In [35]:
# guardamos el dataframe en un archivo .csv
df_reviews.to_csv('ETL2_users_reviews.csv', index= False)

In [ ]:
# guardamos el dataframe en un archivo .parquet
pq.write_table(pa.Table.from_pandas(df_reviews), '../data/ETL2_users_reviews.parquet')